In [8]:
# !pip install simpy

import random
import simpy
import itertools
import collections

RANDOM_SEED = 231
SIM_DURATION = 200

NEW_CUSTOMERS = 15  # Total number of customers
INTERVAL_CUSTOMERS = 3.0  # Generate new customers roughly every x seconds
MIN_PATIENCE = 175  # Min. customer patience
MAX_PATIENCE = 200 # Max. customer patience


def source(env, number, interval, counter):
    # Gera clientes de forma aleatória
    for i in range(number):
        c = customer(env, f'Cliente {i:02d}', counter, time_in_bank=12.0)
        env.process(c)
        t = random.expovariate(1.0 / interval)
        yield env.timeout(t)


def customer(env, name, counter, time_in_bank):
    # Cliente chega, separa suas compras e passa no caixa.
    arrive = env.now
    print(f'🛒 {arrive:7.4f} {name}: *separando os produtos para compra*')
    with counter.request() as req:
        print(f'⏳ {arrive:7.4f} {name}: *entrou na fila do caixa* (tamanho da fila: {len(counter.queue)})')
        patience = random.uniform(MIN_PATIENCE, MAX_PATIENCE)
        results = yield req | env.timeout(patience)
        wait = env.now - arrive

        if req in results:
            print(f'💤 {env.now:7.4f} {name}: esperou {wait:6.3f}s na fila')
            tib = random.expovariate(1.0 / time_in_bank)
            yield env.timeout(tib)
            print(f'✅ {env.now:7.4f} {name}: *finalizou as compras* (tamanho da fila: {len(counter.queue)})')
        else:
            print(f'❌ {env.now:7.4f} {name}: *desistiu em {wait:6.3f}s* (tamanho da fila: {len(counter.queue)})')

print('Caixa de supermercado')
random.seed(RANDOM_SEED)
env = simpy.Environment()

counter = simpy.Resource(env, capacity=1)
env.process(source(env, NEW_CUSTOMERS, INTERVAL_CUSTOMERS, counter))
env.run()

Caixa de supermercado
🛒  0.0000 Cliente 00: *separando os produtos para compra*
⏳  0.0000 Cliente 00: *entrou na fila do caixa* (tamanho da fila: 0)
💤  0.0000 Cliente 00: esperou  0.000s na fila
🛒  5.7829 Cliente 01: *separando os produtos para compra*
⏳  5.7829 Cliente 01: *entrou na fila do caixa* (tamanho da fila: 1)
🛒 11.6864 Cliente 02: *separando os produtos para compra*
⏳ 11.6864 Cliente 02: *entrou na fila do caixa* (tamanho da fila: 2)
🛒 15.8850 Cliente 03: *separando os produtos para compra*
⏳ 15.8850 Cliente 03: *entrou na fila do caixa* (tamanho da fila: 3)
🛒 16.6745 Cliente 04: *separando os produtos para compra*
⏳ 16.6745 Cliente 04: *entrou na fila do caixa* (tamanho da fila: 4)
🛒 19.2026 Cliente 05: *separando os produtos para compra*
⏳ 19.2026 Cliente 05: *entrou na fila do caixa* (tamanho da fila: 5)
✅ 20.7538 Cliente 00: *finalizou as compras* (tamanho da fila: 5)
💤 20.7538 Cliente 01: esperou 14.971s na fila
🛒 21.0342 Cliente 06: *separando os produtos para compra*
